In [1]:
#3/9/2020
#Revision to PO Tracker 

In [2]:
import numpy as np
import pandas as pd
import difflib
import xlsxwriter
import openpyxl
from openpyxl import load_workbook
from openpyxl.chart import LineChart,Reference

In [3]:
#Read in excel file, sheet 1 with appropiate columns
df = pd.read_excel(r"P:\PO Tracker Todd..xlsx",sheet_name=1,usecols="A:B,D:E,G:I,K,N:O")

In [4]:
#Get rows below '~' row
index = df.index[df[' Value'] == '~'] #Returns tuple with index of first instance where'Value' column == '~'
df = df.iloc[index[0]+1:]
#Get rid of NaN filled rows --> At bottom of dataframe
df.dropna(thresh=6,inplace=True)

In [5]:
#Clear any rows with canceled PO's
search = ['Canceled','Closed','No Bid','Denied','Cannot','Test 2','Test','Pay Off','sigs','ppwk']
data = df[~df['Notes'].str.contains('|'.join(search),case=False)].copy()

In [6]:
#Check column data types
data.dtypes

Notes                                  object
Goods/Service / Change Order/Rental    object
Suppliers                              object
Supplier #                             object
Owner                                  object
Owners Representative                  object
Start Date                             object
 Value                                 object
REQ                                    object
PO                                     object
dtype: object

In [7]:
#Check for Nan values
data.isnull().sum()

Notes                                   0
Goods/Service / Change Order/Rental     0
Suppliers                               0
Supplier #                              2
Owner                                   1
Owners Representative                   2
Start Date                              1
 Value                                 14
REQ                                     3
PO                                      3
dtype: int64

In [8]:
#Attempt to find any values in 'Start Date' column that are not dates
#Assuming datetime length is 18 characters long --> Current dtype is object 
mask = data['Start Date'].apply(lambda x: len(str(x))) < 18 

#Get values using boolean mask in 'Start Date' column
#Basically Boolean indexing for values in 'Start Date''
index = data.loc[mask, 'Start Date']

#Turn index to a list and replace those values with ffill values 
data['Start Date'].replace(to_replace=index.tolist(),method='ffill',inplace=True)

#Convert to date time
data['Start Date'] =  pd.to_datetime(data['Start Date'], infer_datetime_format=True)

In [9]:
#Dates that are mistyped to 1900 instead of 2019
time = pd.Series(['2019/01/02']*4)
timereplace = pd.to_datetime(time,infer_datetime_format=True)

#Create another mask to replace these values (4 in total)
mask2 = data['Start Date'] <= '01/01/2018'
index2 = data.loc[mask2, 'Start Date']

#Turn index2 to a list like before and replace those values with timereplace(?) 
data['Start Date'].replace(to_replace=index2.tolist(),value=timereplace,inplace=True)

In [10]:
#Want to see NaN rows in Owner and Owners Rep columns ---> Used Later
data[(data['Owner'].isnull()) | (data['Owners Representative'].isnull()) | (data['Owner'].str.isspace() == True)]

,Notes,Goods/Service / Change Order/Rental,Suppliers,Supplier #,Owner,Owners Representative,Start Date,Value,REQ,PO
1651,Done 8.1.19,Goods,Control Technologies,101050,,Marc Gould,2019-07-31,46571.5,10003131,4500002384
1725,Done 7.11.19,Goods,Carson Valley Golf,101118,NaN,NaN,2019-07-11,2166.85,10003053,4500002319
2434,Done 2.7.19,Goods,Grainger,100106,Brent Stierwalt,NaN,2019-02-06,373.58,10002326,4500001714


In [11]:
#Luckily only a few enteries are missing Owner and Owners Represenative values

grid = data.index[(data['Owner'].isnull()) | (data['Owners Representative'].isnull()) | (data['Owner'].str.isspace() == True)]

data.loc[grid.tolist()[1],'Owner'] = 'Nevada Copper'
data.loc[grid.tolist()[1],'Owners Representative'] = 'Nevada Copper'
data.loc[grid.tolist()[2],'Owners Representative'] = 'Brent Stierwalt'
data.loc[grid.tolist()[0],'Owner'] = 'Marc Gould'

In [12]:
#Value column has blanks and text(?) --> Covert to numeric, else change to NaN value
#The default return dtype is float64 or int64 depending on the data supplied 
#Use the downcast parameter to obtain other dtypes
data[' Value'] = data[' Value'].apply(pd.to_numeric, errors='coerce')

In [13]:
#Check again for null values
data[' Value'].isnull().sum() # There are NaN values --> Currently 16

16

In [14]:
#Replace list
replace = [13963,119.43,122453.03,35000,182519.66,18102.50,453.05,9900,641032.57,82914.40,0,0,56700,534.33,794.51,21727.85]

def Reverse(lst):
    """Reverse order of list"""
    lst.reverse()
    return lst

np_replace = np.array(Reverse(replace), dtype='float64')

In [15]:
#Replace Nan values with list of retrived PO values
data.loc[data[' Value'].isnull(),' Value'] = np_replace

In [16]:
#Finally remove any possible negative values

data = data[~(data[' Value'] <= 0)]

In [17]:
#Look at Owners column --> Fuzzy Matchup

unique = data['Owner'].unique()

In [18]:
unique

array(['David Madsen', 'Larry Brown', 'Lara sims', 'Terry Weiss',
       "Beverly O'Malley", 'August Greth', 'Lumin Ma', 'Tom Sullivan',
       'Ben Morin', 'Michael Bozarth', 'Marc Gould', 'Eugene Bock',
       'Mark Hanley', 'Mariah Joyner', 'Tim Dake', 'Reid Yano',
       'Scott Lawson', 'Lisa Heinz', 'Nick Atiemo', 'Randy Berry',
       'Ian Larkins', 'Tim Dyhr', 'Jennelle Hanson', 'Pamela Moyo',
       'David Swisher', 'Dave Sabourin', 'randy Berry',
       'Shane Eisenbarth', 'John Evans', 'Aubree Barnum',
       'Michael Boomgaarden', 'Greg French', 'Samantha Evans',
       'Clint Schofield', 'Randy Berry ', 'Mark Childs',
       'Korin Carpenter', 'Lisa Heing', 'Lara Sims', 'Nylah Kachelmeyer',
       'Mark Wall', 'William Bond', 'Martin Lopez', 'Jodi Lemos',
       'Jerod Eastman', 'Jenna Joyner', 'Chris Bistolas', 'Thomas Bagan',
       'Nevada Copper', 'Scott Frerichs', 'mark hanley', 'Mark hanley',
       'Brent Stierwalt', 'Kevin Crandall', 'Jason Blais', 'mark Hanley',
  

In [19]:
#Names used for replacement 
names = ['Randy Berry','Samantha Evans','Mark Hanley','Thomas Bagan','Lisa Heinz','Lara Sims']

#Fuzzy MatchUp
closetmatch = [difflib.get_close_matches(options, unique,n=len(unique),cutoff=.75) for options in names]

In [20]:
closetmatch

[['Randy Berry', 'Randy Berry ', 'randy Berry'],
 ['Samantha Evans', 'Samanatha Evans'],
 ['Mark Hanley',
  'mark Hanley',
  'Mark hanley',
  'Mark Hanley ???',
  'mark hanley'],
 ['Thomas Bagan', 'Tom Bagan'],
 ['Lisa Heinz', 'Lisa Heing'],
 ['Lara Sims', 'Lara sims']]

In [21]:
#Create a list of dictionaries mapping to correct name for that particular person
dictlist = [dict(zip(closetmatch[i],[names[i]] * len(closetmatch[i]))) for i,_ in enumerate(closetmatch)]

In [22]:
dictlist

[{'Randy Berry': 'Randy Berry',
  'Randy Berry ': 'Randy Berry',
  'randy Berry': 'Randy Berry'},
 {'Samantha Evans': 'Samantha Evans', 'Samanatha Evans': 'Samantha Evans'},
 {'Mark Hanley': 'Mark Hanley',
  'mark Hanley': 'Mark Hanley',
  'Mark hanley': 'Mark Hanley',
  'Mark Hanley ???': 'Mark Hanley',
  'mark hanley': 'Mark Hanley'},
 {'Thomas Bagan': 'Thomas Bagan', 'Tom Bagan': 'Thomas Bagan'},
 {'Lisa Heinz': 'Lisa Heinz', 'Lisa Heing': 'Lisa Heinz'},
 {'Lara Sims': 'Lara Sims', 'Lara sims': 'Lara Sims'}]

In [23]:
finaldict = {}
for dict1 in dictlist:
    finaldict.update(dict1)

In [24]:
finaldict

{'Randy Berry': 'Randy Berry',
 'Randy Berry ': 'Randy Berry',
 'randy Berry': 'Randy Berry',
 'Samantha Evans': 'Samantha Evans',
 'Samanatha Evans': 'Samantha Evans',
 'Mark Hanley': 'Mark Hanley',
 'mark Hanley': 'Mark Hanley',
 'Mark hanley': 'Mark Hanley',
 'Mark Hanley ???': 'Mark Hanley',
 'mark hanley': 'Mark Hanley',
 'Thomas Bagan': 'Thomas Bagan',
 'Tom Bagan': 'Thomas Bagan',
 'Lisa Heinz': 'Lisa Heinz',
 'Lisa Heing': 'Lisa Heinz',
 'Lara Sims': 'Lara Sims',
 'Lara sims': 'Lara Sims'}

In [25]:
#Create copy
consolidate = data.copy()

#Replace variant of different names with Original 
consolidate['Owner'].replace(to_replace=finaldict,inplace=True)

In [26]:
consolidate.reset_index(drop=True)

,Notes,Goods/Service / Change Order/Rental,Suppliers,Supplier #,Owner,Owners Representative,Start Date,Value,REQ,PO
0,Done 3.30.20,Service,TJA Cvonsulting,101331,David Madsen,David Madsen,2020-03-26,71460.00,10004820,4500003899
1,Done 3.27.20,Goods,Smith Steel,300203,Larry Brown,Larry Brown,2020-03-27,5150.00,10004844,4500002906
2,Done 3.27.20,Goods,Amazon,100498,Lara Sims,Mark Wall,2020-03-27,12464.83,10004838,4500003895
3,Done 3.27.20,Goods,PDM Steel,100212,Terry Weiss,Michael Boomgaarden,2020-03-27,8998.27,10004841,4500003896
4,Done 3.27.20,Service,Cube Consulting,300229,Beverly O'Malley,Beverly O'Malley,2020-03-27,14350.00,10004840,4500003894
...,...,...,...,...,...,...,...,...,...,...
2536,Done 12.17.18,Goods,My Office Products . Com,100183,Jenna Joyner,Jenna Joyner,2018-01-19,1126.39,10002128,4500001546
2537,Done 12.17.18,Goods,Western Nevada,100301,Mark Hanley,Mark Hanley,2018-12-18,49.76,10002126,4500001544
2538,Done 12.17.18,Goods,Sticks and Stones,100546,Mark Hanley,Mark Hanley,2018-12-18,184.68,10002127,4500001545
2539,Done 12.17.18,Goods,Kimball Midwest,100648,Terry Weiss,Terry Weiss,2018-12-20,576.00,10002122,4500001539


In [27]:
#Pivot table to show by month per person

#Grouper resamples by month for every 'Owner' for that month
#Aggregate is straightforward
pivot = consolidate.groupby([pd.Grouper(key='Start Date',freq='M'),'Owner']).agg({"PO":'count'," Value":np.sum})

In [28]:
finalP = pivot.loc['2020-01-31':]

In [29]:
finalP

PO       Value
Start Date Owner                           
2020-01-31 August Greth      12   166888.07
           Ben Morin          1    53728.86
           Beverly O'Malley   1     9000.00
           Clint Schofield    5    73843.49
           David Madsen      44   730098.97
...                          ..         ...
2020-03-31 Shane Eisenbarth   1    10408.67
           Terry Weiss       96  1166050.99
           Tim Dake           1      287.75
           Tim Dyhr           1     2625.00
           Tom Sullivan      24  3626823.61

[85 rows x 2 columns]

In [30]:
#Different ways to look at the data
consolidategroup = consolidate.resample('M',on='Start Date').agg({"PO":'count'," Value":np.sum})

In [31]:
finalC = consolidategroup.loc['2020-01-31':]

In [32]:
finalC

,PO,Value
Start Date,,
2020-01-31,248,7212857.52
2020-02-29,201,4754353.12
2020-03-31,317,8312741.05


In [33]:
#Used to generate initial Excel Sheet

path = r"C:\Users\gcarlstedt\Documents\Work - Nevada Copper\Tom Sullivan\PO Tracker\Supply Chain BPR Graphs\BPR Data.xlsx"

#Create a Pandas Excel writer using XlsxWriter as the engine
with pd.ExcelWriter(path,engine = 'xlsxwriter') as writer:
    finalC.to_excel(writer, sheet_name = 'Total') #Convert df to an XlsxWriter Excel Object
    finalP.to_excel(writer, sheet_name = 'Total by Person by Month') #Convert df to an XlsxWriter Excel Object

writer.save()

In [35]:
wb = load_workbook(path)
ws = wb['Total']

c1 = LineChart()
c1.title = "Cost of PO's by Month"

d1 = Reference(ws,min_col=3,min_row=2,max_col=3,max_row=(len(finalC)+1))
c1.add_data(d1)

c2 = LineChart()
c2.title = "Number of PO's by Month"

d2 = Reference(ws,min_col=2,min_row=2,max_col=2,max_row=(len(finalC)+1))
c2.add_data(d2)

ws.add_chart(c1,'D2')
ws.add_chart(c2,'M2')


wb.save(path)